In [1]:
import os
import numpy as np
import cv2                 
from random import shuffle
from tqdm import tqdm  
import tensorflow as tf 
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau , ModelCheckpoint
from collections import Counter

In [2]:
tf.test.is_gpu_available()

In [8]:
TrianImage="../input/chest-xray-pneumonia/chest_xray/train/" #/content/input/COVID-19/Train
TestImage="../input/chest-xray-pneumonia/chest_xray/test/"   #../input/chest-xray-pneumonia/  ../input/chest-xray-pneumonia/
ValImage="../input/chest-xray-pneumonia/chest_xray/val/"
Normalimages = os.listdir(TrianImage + "/NORMAL")
Pneumonaimages = os.listdir(TrianImage + "/PNEUMONIA") #../input/chest-xray-pneumonia/

In [9]:
print(len(Normalimages), len(Pneumonaimages))
NUM_TRAINING_IMAGES = len(Normalimages) + len(Pneumonaimages)
print(NUM_TRAINING_IMAGES)

In [10]:
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(plt.imread(os.path.join(TrianImage + "/NORMAL",Normalimages[i])),cmap='gray')
    plt.title("NORMAL")
plt.show()

In [11]:
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(plt.imread(os.path.join(TrianImage + "/PNEUMONIA",Pneumonaimages[i])),cmap='gray')
    plt.title("PNEUMONIA")
plt.show()

In [12]:
filepath = [] #list of paths of images of train & validation
categories = [] #label

In [13]:
#Load the images path and label of the training set
filenames = os.listdir(os.path.join(TrianImage,'NORMAL'))
for filename in filenames:
        filepath.append(os.path.join(TrianImage,'NORMAL',filename))
        categories.append("NORMAL") #0: Normal

filenames = os.listdir(os.path.join(TrianImage,'PNEUMONIA'))
for filename in filenames:
        filepath.append(os.path.join(TrianImage,'PNEUMONIA',filename))
        categories.append("PNEUMONIA") #1: Pneumonia

In [14]:
#Load the images path and label of the validation set
filenames = os.listdir(os.path.join(ValImage,'NORMAL'))
for filename in filenames:
        filepath.append(os.path.join(ValImage,'NORMAL',filename))
        categories.append("NORMAL") #0: Normal
        
filenames = os.listdir(os.path.join(ValImage,'PNEUMONIA'))
for filename in filenames:
        filepath.append(os.path.join(ValImage,'PNEUMONIA',filename))
        categories.append("PNEUMONIA") #1: Pneumonia

In [15]:
#Let's create a dataframe containing all images of tRAING and validation set. It's composed by 3875 imageslabelled as "Pneumonia" and 
#1341 images labelled as "Normal".
import pandas as pd
import seaborn as sns
df = pd.DataFrame({'filepath':filepath,'label':categories})
df.info
plot = sns.countplot(x ='label', data = df).set_title('Train+Validation')
print("Df Train-Vali: ",df.head())
print("")
print(df['label'].value_counts())

In [16]:
import re
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

!pip install visualkeras #https://github.com/paulgavrikov/visualkeras
import visualkeras

try: #Part of original notebook to set TPU
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)
    
print(tf.__version__)

In [17]:
train, vali = train_test_split(df, test_size=0.15,stratify=df['label'],random_state=6) #stratify to keep distribution

In [18]:
plot_train = sns.countplot(x ='label', data = train).set_title('Train 85%')
print('Train 85%: ')
print(train['label'].value_counts())

In [19]:
plot_vali = sns.countplot(x ='label', data = vali).set_title('Validation 15%')
print('Validation 15%: ')
print(vali['label'].value_counts())

In [20]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32 * strategy.num_replicas_in_sync #GPU:1 num_replicas
IMAGE_SIZE = [224, 224]
EPOCHS = 40 #early stops in model.fit-->need to save best weights

In [21]:
load = ImageDataGenerator() #To load images

#Load images [0 255] and resize images

train_set = load.flow_from_dataframe(train,
                                        x_col = 'filepath',
                                        y_col = 'label',
                                        target_size = IMAGE_SIZE,
                                        batch_size = BATCH_SIZE,
                                        class_mode = 'binary')

vali_set = load.flow_from_dataframe(vali,
                                        x_col = 'filepath',
                                        y_col = 'label',
                                        target_size = IMAGE_SIZE,
                                        batch_size = BATCH_SIZE,
                                        class_mode = 'binary')

test_set = load.flow_from_directory(TestImage,
                                        target_size = IMAGE_SIZE,
                                        batch_size = BATCH_SIZE,
                                        shuffle = False, #for confusionMatrix
                                        class_mode = 'binary')

#Check classes in output
print("train: ",train_set.class_indices)
print("vali: ",vali_set.class_indices)
print("test: ",test_set.class_indices)
#it returns a DataFrameIterator yielding tuples of (x, y) where x is a numpy array containing a batch of images with shape (batch_size, *target_size)

In [22]:
VAL_IMG_COUNT = vali_set.samples
TEST_IMG_COUNT = test_set.samples

In [23]:
print('Train: ',Counter(train_set.classes))
print('Validation: ',Counter(vali_set.classes))
print('Test: ',Counter(test_set.classes))

In [24]:
plt.figure(figsize=(10,10))
plt.suptitle('Original training set')
for n in range(16):
    ax = plt.subplot(4,4,n+1)
    plt.imshow(train_set[0][0][n].astype(np.uint8))  #uint8 to plot
    if train_set[0][1][n]:
        plt.title("PNEUMONIA")
    else:
        plt.title("NORMAL")
    plt.axis("off")

In [25]:
#Data augmentation only on the training set
augment_gen = ImageDataGenerator(zoom_range = 0.2,
                                   rotation_range=0.2,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   ) #to perform augmentation on training set

In [26]:
aug_train_set =  augment_gen.flow_from_dataframe(train,
                                        x_col = 'filepath',
                                        y_col = 'label',
                                        target_size = IMAGE_SIZE,
                                        batch_size = BATCH_SIZE,
                                        class_mode = 'binary')
print("\n Aug_train: ",train_set.class_indices)

In [27]:
# images after data augmentation
plt.figure(figsize=(10,10))
plt.suptitle('Augmented training set')
for n in range(16):
    ax = plt.subplot(4,4,n+1)
    plt.imshow(aug_train_set[0][0][n].astype(np.uint8))
    if train_set[0][1][n]:
        plt.title("PNEUMONIA")
    else:
        plt.title("NORMAL")
    plt.axis("off")

In [28]:
#class imbalance in the training
counter = Counter(aug_train_set.classes)
#print(counter.items()) #0: Normal, 1: Pneumonia

counter['NORMAL'] = counter.pop(0)
counter['PNEUMONIA'] = counter.pop(1) #change name for visualization dictionary

print('Aug_train_set')
for i in counter:
    print(i,': ',counter[i])

plt.bar(counter.keys(),counter.values())
plt.title('Aug_Train_set')
plt.show()

In [29]:
#correct data imbalancing
COUNT_PNEUMONIA=counter['PNEUMONIA']
COUNT_NORMAL=counter['NORMAL']
TRAIN_IMG_COUNT=aug_train_set.samples

weight_for_0 = (1 / COUNT_NORMAL)*(TRAIN_IMG_COUNT)/2.0 
weight_for_1 = (1 / COUNT_PNEUMONIA)*(TRAIN_IMG_COUNT)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
#image labelled as Normal will be weighted more to balance the data

In [30]:
keras.backend.clear_session() #clear any model

In [31]:
def conv_block(filters):## convolution blockkk
    block = tf.keras.Sequential([
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.SeparableConv2D(filters, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPool2D()
    ]
    )
    
    return block
def dense_block(units, dropout_rate):   ## dense blockk
    block = tf.keras.Sequential([
        tf.keras.layers.Dense(units, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(dropout_rate)
    ])
    
    return block

In [32]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
        
        tf.keras.layers.Rescaling(scale=1./255), #Scaling images between [0 1]
        
        tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
        tf.keras.layers.Conv2D(64, 3, activation='relu', padding='same'),
        tf.keras.layers.MaxPool2D(),
        
        conv_block(128),
        conv_block(256),
        
        conv_block(512),
        tf.keras.layers.Dropout(0.2), 
        
        conv_block(1024),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Flatten(),
        dense_block(2048, 0.7),
        dense_block(512, 0.5),
        dense_block(256, 0.3),
        
        tf.keras.layers.Dense(1, activation='sigmoid') 
    ])

    return model

In [33]:
with strategy.scope():
    model = build_model()

    METRICS = [
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss='binary_crossentropy',
        metrics=METRICS
    )
    
model.summary()

In [34]:
visualkeras.layered_view(model, legend=True)

In [35]:
print("steps_per_epoch : ",TRAIN_IMG_COUNT // BATCH_SIZE)
print("validation_steps : ",VAL_IMG_COUNT // BATCH_SIZE)

In [36]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("CNN_xray_model.h5",
                                                    save_best_only=True,verbose=1)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1)#check val_loss

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0.00001,verbose=1) #if val_loss not improved for patience_epochs-->reduce the learning rate

In [37]:
from sklearn import metrics
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score, 
                             classification_report, f1_score, average_precision_score, precision_recall_fscore_support)

In [38]:
history = model.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[checkpoint_cb,early_stopping_cb,reduce_lr]
)
y_pred_prob_lr = model.predict(test_set)
y_pred_prob_lr = y_pred_prob_lr > 0.5
fpr_lr, tpr_lr, thresholds_lr = roc_curve(test_set.labels, y_pred_prob_lr)
roc_auc_lr = auc(fpr_lr, tpr_lr)
precision_lr, recall_lr, th_lr = precision_recall_curve(test_set.labels, y_pred_prob_lr)

In [39]:
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
    ax[i].plot(history.history[met])
    ax[i].plot(history.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [40]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(13,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="val");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and val Set - Metric acuuracy")
axarr[1].set_title("Training and val Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [41]:
loss, acc, prec, rec = model.evaluate(test_set)

In [42]:
#Predicted labels
predictions = model.predict(test_set)
predictions = predictions > 0.5

#True labels
orig = test_set.labels

In [43]:
cm = confusion_matrix(orig, predictions)
print('Confusion matrix:')
print(cm)
print('')

cr = classification_report(orig, predictions)
print('Classification report:')
print(cr)
print('')

In [44]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=['NORMAL','PNEUMONIA'])
disp.plot()

In [45]:
from sklearn.metrics import accuracy_score
#Accuracy Result
acc = accuracy_score(test_set.labels, predictions)
#Precision, Recall and F-Score (For the whole dataset)
results_all = precision_recall_fscore_support(test_set.labels, predictions, average='macro',zero_division = 1)
#Precision, Recall and F-Score (For each Class)
results_class = precision_recall_fscore_support(test_set.labels, predictions, average=None, zero_division = 1)

#Organise the Results into a Dataframe
metric_columns = ['Precision','Recall', 'F-Score','S']
all_df = pd.concat([pd.DataFrame(list(results_class)).T,pd.DataFrame(list(results_all)).T])
all_df.columns = metric_columns
all_df.index = ['NORMAL', 'PNEUMONA','Total']

#Function to plot the metrics into a nice bar Plot
def metrics_plot(df,metric):
    plt.figure(figsize=(22,10))
    ax = sns.barplot(data =df, x=df.index, y = metric,palette = "Blues_d")
    #Bar Labels
    for p in ax.patches:
        ax.annotate("%.1f%%" % (100*p.get_height()), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
        ha='center', va='bottom', color='black', xytext=(-3, 5),rotation = 'horizontal',textcoords='offset points')
    sns.despine(top=True, right=True, left=True, bottom=False)
    ax.set_xlabel('Class',fontsize = 14,weight = 'bold')
    ax.set_ylabel(metric,fontsize = 14,weight = 'bold')
    ax.set(yticklabels=[])
    ax.axes.get_yaxis().set_visible(False) 
    plt.title(metric+ ' Results per Class', fontsize = 16,weight = 'bold');
    
metrics_plot(all_df, 'Precision')#Results by Class
metrics_plot(all_df, 'Recall')#Results by Class
metrics_plot(all_df, 'F-Score')#Results by Class
print('**Overall Results**')
print('Accuracy Result: %.2f%%'%(acc*100)) #Accuracy of the whole Dataset
print('Precision Result: %.2f%%'%(all_df.iloc[2,0]*100))#Precision of the whole Dataset
print('Recall Result: %.2f%%'%(all_df.iloc[2,1]*100))#Recall of the whole Dataset
print('F-Score Result: %.2f%%'%(all_df.iloc[2,2]*100))#F-Score of the whole Dataset

In [46]:
err=[i for i, (x, y) in enumerate(zip(predictions, orig)) if x != y] #check where orig and predictions don't match (wrong classification)
#print(err)
print('errors: ',len(err))

plt.figure(figsize=(10,10))
plt.suptitle("WRONG PREDICTIONS")
for n in range(16): #plot max 16 wrong predictions
    bn=err[n]//BATCH_SIZE #number of batch of wrong prediction train_set[bn][0][image]
    ib=err[n]-BATCH_SIZE*bn #number of the image in the batch size train_set[bn][0][ib]
    #print('id_err:',err[n],' batch:',bn,' diff:',err[n]-BATCH_SIZE*bn)
    ax = plt.subplot(4,4,n+1)
    plt.imshow(test_set[bn][0][ib].astype(np.uint8))  #uint8 to plot
    
    if test_set[bn][1][ib]: #real value
        plt.title("Real: PNEUMONIA")
    else:
        plt.title("Real: NORMAL")
    plt.axis("off")

In [47]:
# https://keras.io/examples/vision/grad_cam/
from tensorflow import keras

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = keras.Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap, top_pred_index.numpy()

In [48]:
# https://keras.io/examples/vision/grad_cam/
def superimposed_img(image, heatmap):
    # We rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # We use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # We use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # We create an image with RGB colorized heatmap  IMAGE_SIZE[0], IMAGE_SIZE[1]
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((IMAGE_SIZE[0], IMAGE_SIZE[1]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.4 + image
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    return superimposed_img

In [49]:
# label smoothing https://www.linkedin.com/pulse/label-smoothing-solving-overfitting-overconfidence-code-sobh-phd/
def categorical_smooth_loss(y_true, y_pred, label_smoothing=0.1):
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred, label_smoothing=label_smoothing)
    return loss

In [50]:
!pip install efficientnet
import efficientnet.tfkeras as efn

In [51]:
# Data preprocessing
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from glob import glob
from PIL import Image
import os
import random
import cv2
#Model
import keras
from keras.models import Sequential, Model,load_model
from keras.layers import Activation,Dense, Dropout, Flatten, Convolution2D, MaxPooling2D,MaxPool2D,AveragePooling2D,GlobalMaxPooling2D
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
#from tensorflow.keras.layers.normalization import BatchNormalization
from keras.layers.normalization import batch_normalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator,array_to_img
from keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint
from keras.metrics import PrecisionAtRecall,Recall 
#Model Analysis
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from numpy.random import seed
seed(0)
import tensorflow
tensorflow.random.set_seed(0)

In [52]:
keras.backend.clear_session() #delete the previous model (clear model variable)

In [53]:
base_model = tf.keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False)  # Do not include the ImageNet classifier at the top

In [54]:
base_model.trainable = False #freeze the base model

In [55]:
inputs = tf.keras.layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

#Preprocess for VGG16
x = tf.keras.applications.vgg16.preprocess_input(inputs)

# We make sure that the base_model is running in inference mode here, passing `training=False`
x = base_model(x, training=False) #-->not updating weights of this part of the model

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = tf.keras.layers.GlobalAveragePooling2D(name="GlobalAveragePooling2D_head_vgg")(x)

x = tf.keras.layers.Dense(512, activation='relu', name="dense_head_vgg")(x)
x = tf.keras.layers.Dropout(0.2, name="dropout_head_vgg")(x)  # Regularize with dropout

# A Dense classifier with a single unit (binary classification)
outputs = tf.keras.layers.Dense(1, activation='sigmoid', name="predictions_head_vgg")(x)
model_VGG16 = tf.keras.Model(inputs, outputs)

model_VGG16.summary()

In [56]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1) #check val_loss

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=1e-5 ,verbose=1) #min_lr as input for fine tuning next

In [57]:
model_VGG16.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history = model_VGG16.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)

In [58]:
last_conv_layer_name = "conv5_block16_concatination"
classifier_layer_names = [
    "bn",
    "relu",
    "GlobalAveragePooling2D_head_vgg",  #GlobalAveragePooling2D_head_vgg
    "dense_head_vgg",
    "dropout_head_vgg",     #predictions_head_vgg
    "predictions_head_vgg"
]

In [59]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(13,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="val");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and val Set - Metric acuuracy")
axarr[1].set_title("Training and val Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [60]:
# Unfreeze the base model
base_model.trainable = True

model_VGG16.summary()

model_VGG16.compile(optimizer=tf.keras.optimizers.Adam(1e-5), #Very low learning rate
              loss='binary_crossentropy',
              metrics='accuracy')

In [61]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("VGG16_xray_model.h5",
                                                    save_best_only=True,verbose=1)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1) #check val_loss by default for the early stop (check if impro

In [62]:
history = model_VGG16.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)
#y_pred = model_MobileNet.predict(test_set)
y_pred_prob_VGG16 = model_VGG16.predict(test_set)
y_pred_prob_VGG16 = y_pred_prob_VGG16 > 0.5
fpr_VGG16, tpr_VGG16, thresholds_VGG16 = roc_curve(test_set.labels, y_pred_prob_VGG16)
roc_auc_VGG16 = auc(fpr_VGG16, tpr_VGG16)
precision_VGG16, recall_VGG16, th_VGG16 = precision_recall_curve(test_set.labels, y_pred_prob_VGG16)

In [63]:
model_VGG16.save("model_VGG16.h5")

In [64]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(15,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="vali");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and val Set - Metric acuuracy")
axarr[1].set_title("Training and val Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots for Mobilenet model',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [65]:
loss, acc = model_VGG16.evaluate(test_set)

In [66]:
#Predicted labels
prediction_VGG16 = model_VGG16.predict(test_set)
prediction_VGG16 = prediction_VGG16 > 0.5

#True labels
orig_VGG16 = test_set.labels

In [67]:
cm_VGG16 = confusion_matrix(orig_VGG16, y_pred_prob_VGG16)
print('Confusion matrix:')
print(cm_VGG16)
print('')

cr_VGG16 = classification_report(orig_VGG16, y_pred_prob_VGG16)
print('Classification report:')
print(cr_VGG16)
print('')

In [68]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_VGG16,display_labels=['NORMAL','PNEUMONIA'])
disp.plot()

In [69]:
err=[i for i, (x, y) in enumerate(zip(y_pred_prob_VGG16, test_set.labels)) if x != y] #check where orig and predictions don't match (wrong classification)
#print(err)
print('errori commessi: ',len(err))

plt.figure(figsize=(10,10))
plt.suptitle("WRONG PREDICTIONS")
for n in range(16): #plot max 16 wrong predictions
    bn=err[n]//BATCH_SIZE #number of batch of wrong prediction train_set[bn][0][image]
    ib=err[n]-BATCH_SIZE*bn #number of the image in the batch size train_set[bn][0][ib]
    #print('id_err:',err[n],' batch:',bn,' diff:',err[n]-BATCH_SIZE*bn)
    ax = plt.subplot(4,4,n+1)
    plt.imshow(test_set[bn][0][ib].astype(np.uint8))  #uint8 to plot
    
    if test_set[bn][1][ib]: #real value
        plt.title("Real: PNEUMONIA")
    else:
        plt.title("Real: NORMAL")
    plt.axis("off")

In [70]:
#Accuracy Result
acc = accuracy_score(test_set.labels, y_pred_prob_VGG16)
#Precision, Recall and F-Score (For the whole dataset)
results_all = precision_recall_fscore_support(test_set.labels, y_pred_prob_VGG16, average='macro',zero_division = 1)
#Precision, Recall and F-Score (For each Class)
results_class = precision_recall_fscore_support(test_set.labels, y_pred_prob_VGG16, average=None, zero_division = 1)

#Organise the Results into a Dataframe
metric_columns = ['Precision','Recall', 'F-Score','S']
all_df = pd.concat([pd.DataFrame(list(results_class)).T,pd.DataFrame(list(results_all)).T])
all_df.columns = metric_columns
all_df.index = ['NORMAL', 'PNEUMONIA', 'Total']

#Function to plot the metrics into a nice bar Plot
def metrics_plot(df,metric):
    plt.figure(figsize=(22,10))
    ax = sns.barplot(data =df, x=df.index, y = metric,palette = "Blues_d")
    #Bar Labels
    for p in ax.patches:
        ax.annotate("%.1f%%" % (100*p.get_height()), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
        ha='center', va='bottom', color='black', xytext=(-3, 5),rotation = 'horizontal',textcoords='offset points')
    sns.despine(top=True, right=True, left=True, bottom=False)
    ax.set_xlabel('Class',fontsize = 14,weight = 'bold')
    ax.set_ylabel(metric,fontsize = 14,weight = 'bold')
    ax.set(yticklabels=[])
    ax.axes.get_yaxis().set_visible(False) 
    plt.title(metric+ ' Results per Class', fontsize = 16,weight = 'bold');
    
metrics_plot(all_df, 'Precision')#Results by Class
metrics_plot(all_df, 'Recall')#Results by Class
metrics_plot(all_df, 'F-Score')#Results by Class
print('**Overall Results**')
print('Accuracy Result: %.2f%%'%(acc*100)) #Accuracy of the whole Dataset
print('Precision Result: %.2f%%'%(all_df.iloc[2,0]*100))#Precision of the whole Dataset
print('Recall Result: %.2f%%'%(all_df.iloc[2,1]*100))#Recall of the whole Dataset
print('F-Score Result: %.2f%%'%(all_df.iloc[2,2]*100))#F-Score of the whole Dataset

In [71]:
keras.backend.clear_session()

In [72]:
base_model = tf.keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False)  # Do not include the ImageNet classifier at the top

In [73]:
base_model.trainable = False #freeze the base model

In [74]:
inputs = tf.keras.layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

#Preprocess for Xception
x = tf.keras.applications.xception.preprocess_input(inputs)

# We make sure that the base_model is running in inference mode here passing `training=False`  Xception
x = base_model(x, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = tf.keras.layers.GlobalAveragePooling2D(name="GlobalAveragePooling2D_head_xception")(x)

x = tf.keras.layers.Dense(256, activation="relu", name="dense_head_xception")(x)
x = tf.keras.layers.Dropout(0.2, name="dropout_head_xception")(x)  # Regularize with dropout

# A Dense classifier with a single unit (binary classification)
outputs = tf.keras.layers.Dense(1, activation="sigmoid", name="predictions_head_xception")(x)
model_Xception = tf.keras.Model(inputs, outputs)

model_Xception.summary()

In [75]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1)#check val_loss

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=1e-5 ,verbose=1) #min_lr as input for fine tuning next

In [76]:
model_Xception.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history = model_Xception.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)


In [77]:
last_conv_layer_name = "conv5_xception_concatination"
classifier_layer_names = [
    "bn",
    "relu",
    "GlobalAveragePooling2D_head_xception",  #GlobalAveragePooling2D_head_vgg
    "dense_head_xception",
    "dropout_head_xception",     #predictions_head_vgg
    "predictions_head_xception"
]

In [78]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(13,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="val");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and val Set - Metric acuuracy")
axarr[1].set_title("Training and val Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [79]:
# Unfreeze the base model
base_model.trainable = True

model_Xception.summary()

model_Xception.compile(optimizer=tf.keras.optimizers.Adam(1e-5), #Very low learning rate
              loss='binary_crossentropy',
              metrics='accuracy')

In [80]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("Xception_xray_model.h5",
                                                    save_best_only=True,verbose=1)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1) #check val_loss

In [81]:
history = model_Xception.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)
#y_pred = model_MobileNet.predict(test_set)
y_pred_prob_Xception = model_Xception.predict(test_set)
y_pred_prob_Xception = y_pred_prob_Xception > 0.5
fpr_Xception, tpr_Xception, thresholds_Xception = roc_curve(test_set.labels, y_pred_prob_Xception)
roc_auc_Xception = auc(fpr_Xception, tpr_Xception)
precision_Xception, recall_Xception, th_Xception = precision_recall_curve(test_set.labels, y_pred_prob_Xception)


In [82]:
model_Xception.save("model_Xception.h5")

In [83]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(15,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="vali");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and val Set - Metric acuuracy")
axarr[1].set_title("Training and val Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots for Mobilenet model',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [84]:
loss, acc = model_Xception.evaluate(test_set)

In [85]:
#Predicted labels
prediction_Xception = model_Xception.predict(test_set)
prediction_Xception = prediction_Xception > 0.5

#True labels
orig_Xception = test_set.labels

In [86]:
cm_Xception = confusion_matrix(orig_Xception, prediction_Xception)
print('Confusion matrix:')
print(cm_Xception)
print('')

cr_Xception = classification_report(orig_Xception, prediction_Xception)
print('Classification report:')
print(cr_Xception)
print('')

In [87]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_Xception,display_labels=['NORMAL','PNEUMONIA'])
disp.plot()

In [88]:
err=[i for i, (x, y) in enumerate(zip(y_pred_prob_Xception, test_set.labels)) if x != y] #check where orig and predictions don't match (wrong classification)
#print(err)
print('errors: ',len(err))

plt.figure(figsize=(10,10))
plt.suptitle("WRONG PREDICTIONS")
for n in range(16): #plot max 16 wrong predictions
    bn=err[n]//BATCH_SIZE #number of batch of wrong prediction train_set[bn][0][image]
    ib=err[n]-BATCH_SIZE*bn #number of the image in the batch size train_set[bn][0][ib]
    #print('id_err:',err[n],' batch:',bn,' diff:',err[n]-BATCH_SIZE*bn)
    ax = plt.subplot(4,4,n+1)
    plt.imshow(test_set[bn][0][ib].astype(np.uint8))  #uint8 to plot
    
    if test_set[bn][1][ib]: #real value
        plt.title("Real: PNEUMONIA")
    else:
        plt.title("Real: NORMAL")
    plt.axis("off")

In [89]:
#Accuracy Result
acc = accuracy_score(test_set.labels, y_pred_prob_Xception)
#Precision, Recall and F-Score (For the whole dataset)
results_all = precision_recall_fscore_support(test_set.labels, y_pred_prob_Xception, average='macro',zero_division = 1)
#Precision, Recall and F-Score (For each Class)
results_class = precision_recall_fscore_support(test_set.labels, y_pred_prob_Xception, average=None, zero_division = 1)

#Organise the Results into a Dataframe
metric_columns = ['Precision','Recall', 'F-Score','S']
all_df = pd.concat([pd.DataFrame(list(results_class)).T,pd.DataFrame(list(results_all)).T])
all_df.columns = metric_columns
all_df.index = ['NORMAL', 'PNEUMONIA', 'Total']

#Function to plot the metrics into a nice bar Plot
def metrics_plot(df,metric):
    plt.figure(figsize=(22,10))
    ax = sns.barplot(data =df, x=df.index, y = metric,palette = "Blues_d")
    #Bar Labels
    for p in ax.patches:
        ax.annotate("%.1f%%" % (100*p.get_height()), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
        ha='center', va='bottom', color='black', xytext=(-3, 5),rotation = 'horizontal',textcoords='offset points')
    sns.despine(top=True, right=True, left=True, bottom=False)
    ax.set_xlabel('Class',fontsize = 14,weight = 'bold')
    ax.set_ylabel(metric,fontsize = 14,weight = 'bold')
    ax.set(yticklabels=[])
    ax.axes.get_yaxis().set_visible(False) 
    plt.title(metric+ ' Results per Class', fontsize = 16,weight = 'bold');
    
metrics_plot(all_df, 'Precision')#Results by Class
metrics_plot(all_df, 'Recall')#Results by Class
metrics_plot(all_df, 'F-Score')#Results by Class
print('**Overall Results**')
print('Accuracy Result: %.2f%%'%(acc*100)) #Accuracy of the whole Dataset
print('Precision Result: %.2f%%'%(all_df.iloc[2,0]*100))#Precision of the whole Dataset
print('Recall Result: %.2f%%'%(all_df.iloc[2,1]*100))#Recall of the whole Dataset
print('F-Score Result: %.2f%%'%(all_df.iloc[2,2]*100))#F-Score of the whole Dataset

In [90]:
keras.backend.clear_session()

In [91]:
base_model = tf.keras.applications.ResNet152V2(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False)  # Do not include the ImageNet classifier at the top

In [92]:
base_model.trainable = False #freeze the base model

In [93]:
inputs = tf.keras.layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

#Preprocess for ResNet152V2
x = tf.keras.applications.resnet_v2.preprocess_input(inputs)

# We make sure that the base_model is running in inference mode here, passing `training=False`
x = base_model(x, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = tf.keras.layers.GlobalAveragePooling2D(name="Globalaveragepooling2d_head_resnet_v2")(x)

x = tf.keras.layers.Dense(256, activation='relu', name="dense_head_resnet_v2")(x)
x = tf.keras.layers.Dropout(0.2, name="dropout_head_resnet_v2")(x)  # Regularize with dropout

# A Dense classifier with a single unit (binary classification)
outputs = tf.keras.layers.Dense(1, activation='sigmoid', name="predictions_head_resnet_v2")(x)
model_ResNet152V2 = tf.keras.Model(inputs, outputs)

model_ResNet152V2.summary()

In [94]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1)#check val_loss

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=1e-5 ,verbose=1) #min_lr as input for fine tuning next

In [95]:
model_ResNet152V2.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history = model_ResNet152V2.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)

In [96]:
last_conv_layer_name = "top_conv"
classifier_layer_names = [
    "top_bn",
    "top_activation",
    "Globalaveragepooling2d_head_resnet_v2",
    "dense_head_resnet_v2",
    "dropout_head_resnet_v2",
    "predictions_head_resnet_v2"
]

In [97]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(13,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="val");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and validation Set - Metric acuuracy")
axarr[1].set_title("Training and validation Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [98]:
# Unfreeze the base model
base_model.trainable = True

model_ResNet152V2.summary()

model_ResNet152V2.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  # Very low learning rate
              loss='binary_crossentropy',
              metrics='accuracy')

In [99]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("ResNet152V2_xray_model.h5",
                                                    save_best_only=True,verbose=1)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=3,
                                                     restore_best_weights=True,verbose=1)#check val_loss

In [100]:
history = model_ResNet152V2.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)
#y_pred = model_MobileNet.predict(test_set)
y_pred_prob_ResNet152V2 = model_ResNet152V2.predict(test_set)
y_pred_prob_ResNet152V2 = y_pred_prob_ResNet152V2 > 0.5
fpr_ResNet152V2, tpr_ResNet152V2, thresholds_ResNet152V2 = roc_curve(test_set.labels, y_pred_prob_ResNet152V2)
roc_auc_ResNet152V2 = auc(fpr_ResNet152V2, tpr_ResNet152V2)
precision_ResNet152V2, recall_ResNet152V2, th_ResNet152V2 = precision_recall_curve(test_set.labels, y_pred_prob_ResNet152V2)


In [101]:
model_ResNet152V2.save("model_ResNet152V2.h5")

In [102]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(15,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="vali");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and val Set - Metric acuuracy")
axarr[1].set_title("Training and val Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots for Mobilenet model',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [103]:
loss, acc = model_ResNet152V2.evaluate(test_set)

In [104]:
#Predicted labels
prediction_ResNet152V2 = model_ResNet152V2.predict(test_set)
prediction_ResNet152V2 = prediction_ResNet152V2 > 0.5

#True labels
orig_ResNet152V2 = test_set.labels

In [105]:
cm_ResNet152V2 = confusion_matrix(orig_ResNet152V2, prediction_ResNet152V2)
print('Confusion matrix:')
print(cm_ResNet152V2)
print('')

cr_ResNet152V2 = classification_report(orig_ResNet152V2, prediction_ResNet152V2)
print('Classification report:')
print(cr_ResNet152V2)
print('')

In [106]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_ResNet152V2,display_labels=['NORMAL','PNEUMONIA'])
disp.plot()

In [107]:
err=[i for i, (x, y) in enumerate(zip(y_pred_prob_ResNet152V2, test_set.labels)) if x != y] #check where orig and predictions don't match (wrong classification)
#print(err)
print('errors: ',len(err))

plt.figure(figsize=(10,10))
plt.suptitle("WRONG PREDICTIONS")
for n in range(16): #plot max 16 wrong predictions
    bn=err[n]//BATCH_SIZE #number of batch of wrong prediction train_set[bn][0][image]
    ib=err[n]-BATCH_SIZE*bn #number of the image in the batch size train_set[bn][0][ib]
    #print('id_err:',err[n],' batch:',bn,' diff:',err[n]-BATCH_SIZE*bn)
    ax = plt.subplot(4,4,n+1)
    plt.imshow(test_set[bn][0][ib].astype(np.uint8))  #uint8 to plot
    
    if test_set[bn][1][ib]: #real value
        plt.title("Real: PNEUMONIA")
    else:
        plt.title("Real: NORMAL")
    plt.axis("off")

In [108]:
#Accuracy Result
acc = accuracy_score(test_set.labels, y_pred_prob_ResNet152V2)
#Precision, Recall and F-Score (For the whole dataset)
results_all = precision_recall_fscore_support(test_set.labels, y_pred_prob_ResNet152V2, average='macro',zero_division = 1)
#Precision, Recall and F-Score (For each Class)
results_class = precision_recall_fscore_support(test_set.labels, y_pred_prob_ResNet152V2, average=None, zero_division = 1)

#Organise the Results into a Dataframe
metric_columns = ['Precision','Recall', 'F-Score','S']
all_df = pd.concat([pd.DataFrame(list(results_class)).T,pd.DataFrame(list(results_all)).T])
all_df.columns = metric_columns
all_df.index = ['NORMAL', 'PNEUMONIA', 'Total']

#Function to plot the metrics into a nice bar Plot
def metrics_plot(df,metric):
    plt.figure(figsize=(22,10))
    ax = sns.barplot(data =df, x=df.index, y = metric,palette = "Blues_d")
    #Bar Labels
    for p in ax.patches:
        ax.annotate("%.1f%%" % (100*p.get_height()), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
        ha='center', va='bottom', color='black', xytext=(-3, 5),rotation = 'horizontal',textcoords='offset points')
    sns.despine(top=True, right=True, left=True, bottom=False)
    ax.set_xlabel('Class',fontsize = 14,weight = 'bold')
    ax.set_ylabel(metric,fontsize = 14,weight = 'bold')
    ax.set(yticklabels=[])
    ax.axes.get_yaxis().set_visible(False) 
    plt.title(metric+ ' Results per Class', fontsize = 16,weight = 'bold');
    
metrics_plot(all_df, 'Precision')#Results by Class
metrics_plot(all_df, 'Recall')#Results by Class
metrics_plot(all_df, 'F-Score')#Results by Class
print('**Overall Results**')
print('Accuracy Result: %.2f%%'%(acc*100)) #Accuracy of the whole Dataset
print('Precision Result: %.2f%%'%(all_df.iloc[2,0]*100))#Precision of the whole Dataset
print('Recall Result: %.2f%%'%(all_df.iloc[2,1]*100))#Recall of the whole Dataset
print('F-Score Result: %.2f%%'%(all_df.iloc[2,2]*100))#F-Score of the whole Dataset

In [109]:
keras.backend.clear_session()

In [111]:
base_model = tf.keras.applications.InceptionV3(
    weights='imagenet',  # Load weights pre-trained on ImageNet
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
    include_top=False)  # Do not include the ImageNet classifier at the top

In [112]:
base_model.trainable = False #freeze the base model

In [113]:
inputs = keras.layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

x = base_model(inputs, training=False)

x = keras.layers.GlobalAveragePooling2D(name="Globalaveragepooling2d_head_InceptionV3")(x)
x = keras.layers.Dense(256, activation='relu', name="dense_head_InceptionV3")(x)  #dense_head_InceptionV3
x = keras.layers.Dropout(0.2, name="dropout_head_InceptionV3")(x)
outputs = keras.layers.Dense(1, activation='sigmoid', name="predictions_head_InceptionV3")(x)
model_InceptionV3  = keras.Model(inputs, outputs)
model_InceptionV3.summary()

In [114]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1) #check val_loss

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=1e-5 ,verbose=1) #min_lr as input for fine tuning next

In [115]:
model_InceptionV3.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history = model_InceptionV3.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)

In [116]:
last_conv_layer_name = "block5_pool"
classifier_layer_names = [
    "Globalaveragepooling2d_head_InceptionV3",
    "dense_head_InceptionV3",
    "dropout_head_InceptionV3",
    "predictions_head_InceptionV3"
]

In [117]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(13,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="val");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and validation Set - Metric acuuracy")
axarr[1].set_title("Training and validation Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [118]:
# Unfreeze the base model
base_model.trainable = True

model_InceptionV3.summary()

model_InceptionV3.compile(optimizer=tf.keras.optimizers.Adam(1e-5), #Very low learning rate
              loss='binary_crossentropy',
              metrics='accuracy')

In [119]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("InceptionV3_xray_model.h5",
                                                    save_best_only=True,verbose=1)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1) #check val_loss by default for the early stop (check if impro

In [120]:
history = model_InceptionV3.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)
#y_pred = model_MobileNet.predict(test_set)
y_pred_prob_InceptionV3 = model_InceptionV3.predict(test_set)
y_pred_prob_InceptionV3 = y_pred_prob_InceptionV3 > 0.5
fpr_InceptionV3, tpr_InceptionV3, thresholds_InceptionV3 = roc_curve(test_set.labels, y_pred_prob_InceptionV3)
roc_auc_InceptionV3 = auc(fpr_InceptionV3, tpr_InceptionV3)
precision_InceptionV3, recall_InceptionV3, th_InceptionV3 = precision_recall_curve(test_set.labels, y_pred_prob_InceptionV3)


In [121]:
model_InceptionV3.save("model_InceptionV3.h5")

In [122]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(15,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="vali");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="val");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and val Set - Metric acuuracy")
axarr[1].set_title("Training and val Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots for Mobilenet model',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [123]:
loss, acc = model_InceptionV3.evaluate(test_set)

In [124]:
#Predicted labels
prediction_InceptionV3 = model_InceptionV3.predict(test_set)
prediction_InceptionV3 = prediction_InceptionV3 > 0.5

#True labels
orig_InceptionV3 = test_set.labels

In [125]:
cm_InceptionV3 = confusion_matrix(orig_InceptionV3, prediction_InceptionV3)
print('Confusion matrix:')
print(cm_InceptionV3)
print('')

cr_InceptionV3 = classification_report(orig_InceptionV3, prediction_InceptionV3)
print('Classification report:')
print(cr_InceptionV3)
print('')

In [126]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_InceptionV3,display_labels=['NORMAL','PNEUMONIA'])
disp.plot()

In [127]:
err=[i for i, (x, y) in enumerate(zip(y_pred_prob_InceptionV3, test_set.labels)) if x != y] #check where orig and predictions don't match (wrong classification)
#print(err)
print('errors: ',len(err))

plt.figure(figsize=(10,10))
plt.suptitle("WRONG PREDICTIONS")
for n in range(16): #plot max 16 wrong predictions
    bn=err[n]//BATCH_SIZE #number of batch of wrong prediction train_set[bn][0][image]
    ib=err[n]-BATCH_SIZE*bn #number of the image in the batch size train_set[bn][0][ib]
    #print('id_err:',err[n],' batch:',bn,' diff:',err[n]-BATCH_SIZE*bn)
    ax = plt.subplot(4,4,n+1)
    plt.imshow(test_set[bn][0][ib].astype(np.uint8))  #uint8 to plot
    
    if test_set[bn][1][ib]: #real value
        plt.title("Real: NORMAL")
    else:
        plt.title("Real: PNEUMONIA")
    plt.axis("off")

In [128]:
#Accuracy Result
acc = accuracy_score(test_set.labels, y_pred_prob_InceptionV3)
#Precision, Recall and F-Score (For the whole dataset)
results_all = precision_recall_fscore_support(test_set.labels, y_pred_prob_InceptionV3, average='macro',zero_division = 1)
#Precision, Recall and F-Score (For each Class)
results_class = precision_recall_fscore_support(test_set.labels, y_pred_prob_InceptionV3, average=None, zero_division = 1)

#Organise the Results into a Dataframe
metric_columns = ['Precision','Recall', 'F-Score','S']
all_df = pd.concat([pd.DataFrame(list(results_class)).T,pd.DataFrame(list(results_all)).T])
all_df.columns = metric_columns
all_df.index = ['NORMAL', 'PNEUMONIA', 'Total']

#Function to plot the metrics into a nice bar Plot
def metrics_plot(df,metric):
    plt.figure(figsize=(22,10))
    ax = sns.barplot(data =df, x=df.index, y = metric,palette = "Blues_d")
    #Bar Labels
    for p in ax.patches:
        ax.annotate("%.1f%%" % (100*p.get_height()), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
        ha='center', va='bottom', color='black', xytext=(-3, 5),rotation = 'horizontal',textcoords='offset points')
    sns.despine(top=True, right=True, left=True, bottom=False)
    ax.set_xlabel('Class',fontsize = 14,weight = 'bold')
    ax.set_ylabel(metric,fontsize = 14,weight = 'bold')
    ax.set(yticklabels=[])
    ax.axes.get_yaxis().set_visible(False) 
    plt.title(metric+ ' Results per Class', fontsize = 16,weight = 'bold');
    
metrics_plot(all_df, 'Precision')#Results by Class
metrics_plot(all_df, 'Recall')#Results by Class
metrics_plot(all_df, 'F-Score')#Results by Class
print('**Overall Results**')
print('Accuracy Result: %.2f%%'%(acc*100)) #Accuracy of the whole Dataset
print('Precision Result: %.2f%%'%(all_df.iloc[2,0]*100))#Precision of the whole Dataset
print('Recall Result: %.2f%%'%(all_df.iloc[2,1]*100))#Recall of the whole Dataset
print('F-Score Result: %.2f%%'%(all_df.iloc[2,2]*100))#F-Score of the whole Dataset

In [129]:
# model_densenet
# model_efnet
# model_vgg

for layer in model_VGG16.layers:
  layer.trainable = False

for layer in model_Xception.layers:
  layer.trainable = False

for layer in model_ResNet152V2.layers:
  layer.trainable = False

for layer in model_InceptionV3.layers:
  layer.trainable = False

#for layer in model_EfficientNetB7.layers:
#  layer.trainable = False    

#for layer in model_NasNet.layers:
#  layer.trainable = False

visible = tf.keras.layers.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), name="input")
x = model_VGG16(visible)
x = model_Xception(visible)
x = model_ResNet152V2(visible)
x = model_InceptionV3(visible)
#x = model_EfficientNetB7(visible)
merge = tf.keras.layers.concatenate([x, x, x, x], name= "concatallprobs")
model_out = tf.keras.layers.Dense(1, activation='sigmoid', name="predictions")(merge)

model_VGG16_Xception_ResNet152V2_InceptionV3 = Model(inputs=visible, outputs=model_out)
model_VGG16_Xception_ResNet152V2_InceptionV3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])
model_VGG16_Xception_ResNet152V2_InceptionV3.summary()

In [130]:
#Model Parameters
#epochs = 2 # make it 1000

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1) #check val_loss

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=1e-5 ,verbose=1) #min_lr as input for fine tuning next

In [131]:
history = model_VGG16_Xception_ResNet152V2_InceptionV3.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)
#y_pred = model_MobileNet_DenseNet_EfNet.predict(test_set)
y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3 = model_VGG16_Xception_ResNet152V2_InceptionV3.predict(test_set)
y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3 = y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3 > 0.5
fpr_VGG16_Xception_ResNet152V2_InceptionV3, tpr_VGG16_Xception_ResNet152V2_InceptionV3, thresholds_VGG16_Xception_ResNet152V2_InceptionV3 = roc_curve(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3)
roc_auc_VGG16_Xception_ResNet152V2_InceptionV3 = auc(fpr_VGG16_Xception_ResNet152V2_InceptionV3, tpr_VGG16_Xception_ResNet152V2_InceptionV3)
precision_VGG16_Xception_ResNet152V2_InceptionV3, recall_VGG16_Xception_ResNet152V2_InceptionV3, th_VGG16_Xception_ResNet152V2_InceptionV3 = precision_recall_curve(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3)

In [132]:
model_VGG16_Xception_ResNet152V2_InceptionV3.save("model_VGG16_Xception_ResNet152V2_InceptionV3.h5")

In [133]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(13,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="VAL");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="VAL");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and VAL Set - Metric acuuracy")
axarr[1].set_title("Training and VAL Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [134]:
loss, acc = model_VGG16_Xception_ResNet152V2_InceptionV3.evaluate(test_set)

In [135]:
#Predicted labels
prediction_VGG16_Xception_ResNet152V2_InceptionV3 = model_VGG16_Xception_ResNet152V2_InceptionV3.predict(test_set)
prediction_VGG16_Xception_ResNet152V2_InceptionV3 = prediction_VGG16_Xception_ResNet152V2_InceptionV3 > 0.5

#True labels
orig_VGG16_Xception_ResNet152V2_InceptionV3 = test_set.labels

In [136]:
cm_VGG16_Xception_ResNet152V2_InceptionV3 = confusion_matrix(orig_VGG16_Xception_ResNet152V2_InceptionV3, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3)
print('Confusion matrix:')
print(cm_VGG16_Xception_ResNet152V2_InceptionV3)
print('')

cr_VGG16_Xception_ResNet152V2_InceptionV3 = classification_report(orig_VGG16_Xception_ResNet152V2_InceptionV3, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3)
print('Classification report:')
print(cr_VGG16_Xception_ResNet152V2_InceptionV3)
print('')

In [137]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_VGG16_Xception_ResNet152V2_InceptionV3,display_labels=['NORMAL','PNEUMONIA'])
disp.plot()

In [138]:
err=[i for i, (x, y) in enumerate(zip(y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3, test_set.labels)) if x != y] #check where orig and predictions don't match (wrong classification)
#print(err)
print('errori commessi: ',len(err))

plt.figure(figsize=(10,10))
plt.suptitle("WRONG PREDICTIONS")
for n in range(16): #plot max 16 wrong predictions
    bn=err[n]//BATCH_SIZE #number of batch of wrong prediction train_set[bn][0][image]
    ib=err[n]-BATCH_SIZE*bn #number of the image in the batch size train_set[bn][0][ib]
    #print('id_err:',err[n],' batch:',bn,' diff:',err[n]-BATCH_SIZE*bn)
    ax = plt.subplot(4,4,n+1)
    plt.imshow(test_set[bn][0][ib].astype(np.uint8))  #uint8 to plot
    
    if test_set[bn][1][ib]: #real value
        plt.title("Real: PNEUMONIA")
    else:
        plt.title("Real: NORMAL")
    plt.axis("off")

In [139]:
#Accuracy Result
acc = accuracy_score(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3)
#Precision, Recall and F-Score (For the whole dataset)
results_all = precision_recall_fscore_support(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3, average='macro',zero_division = 1)
#Precision, Recall and F-Score (For each Class)
results_class = precision_recall_fscore_support(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3, average=None, zero_division = 1)

#Organise the Results into a Dataframe
metric_columns = ['Precision','Recall', 'F-Score','S']
all_df = pd.concat([pd.DataFrame(list(results_class)).T,pd.DataFrame(list(results_all)).T])
all_df.columns = metric_columns
all_df.index = ['NORMAL', 'PNEUMONIA', 'Total']

#Function to plot the metrics into a nice bar Plot
def metrics_plot(df,metric):
    plt.figure(figsize=(22,10))
    ax = sns.barplot(data =df, x=df.index, y = metric,palette = "Blues_d")
    #Bar Labels
    for p in ax.patches:
        ax.annotate("%.1f%%" % (100*p.get_height()), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
        ha='center', va='bottom', color='black', xytext=(-3, 5),rotation = 'horizontal',textcoords='offset points')
    sns.despine(top=True, right=True, left=True, bottom=False)
    ax.set_xlabel('Class',fontsize = 14,weight = 'bold')
    ax.set_ylabel(metric,fontsize = 14,weight = 'bold')
    ax.set(yticklabels=[])
    ax.axes.get_yaxis().set_visible(False) 
    plt.title(metric+ ' Results per Class', fontsize = 16,weight = 'bold');
    
metrics_plot(all_df, 'Precision')#Results by Class
metrics_plot(all_df, 'Recall')#Results by Class
metrics_plot(all_df, 'F-Score')#Results by Class
print('**Overall Results**')
print('Accuracy Result: %.2f%%'%(acc*100)) #Accuracy of the whole Dataset
print('Precision Result: %.2f%%'%(all_df.iloc[2,0]*100))#Precision of the whole Dataset
print('Recall Result: %.2f%%'%(all_df.iloc[2,1]*100))#Recall of the whole Dataset
print('F-Score Result: %.2f%%'%(all_df.iloc[2,2]*100))#F-Score of the whole Dataset

In [140]:
import cv2
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from tensorflow.keras.callbacks import *

In [141]:
train_set.class_indices
vali_set.class_indices
test_set.class_indices

In [142]:
import numpy as np

def normalize(array2d: np.ndarray) -> np.ndarray:
    return array2d / array2d.sum(axis=1, keepdims=True)

class_names = ["PNEUMONIA", "NORMAL"]
num_classes = len(class_names)
num_samples = 500

# Mock ground truth
ground_truth = np.random.choice(range(num_classes), size=num_samples, p=[0.5, 0.5])

# Mock model predictions
perfect_model = np.eye(num_classes)[ground_truth]
noisy_model = normalize(
    perfect_model + 2 * np.random.random((num_samples, num_classes))
)
random_model = normalize(np.random.random((num_samples, num_classes)))

In [143]:
!pip install metriculous

In [144]:
import metriculous

metriculous.compare_classifiers(
    ground_truth=ground_truth,
    model_predictions=[perfect_model, noisy_model, random_model],
    model_names=["Perfect Model", "Noisy Model", "Random Model"],
    class_names=class_names,
    one_vs_all_figures=True, # This line is important to include ROC curves in the output
).save_html("model_comparison.html").display()

In [145]:
!pip install plot_metric

In [146]:
!pip install ggplot

In [148]:
from tensorflow.keras.models import load_model
def VGG16():
      return (load_model('model_VGG16.h5'))  #model_DenseNet_30_09596.h5   #model_EfNet_30_09596.h5    /content/model_VGG16.h5
def Xception():
      return(load_model('model_Xception.h5'))    #model_MobileNet_30_09596.h5  /content/model_Xception.h5
def ResNet152V2():
      return(load_model('model_ResNet152V2.h5')) #./model_VGG16_Xception_ResNet152V2_InceptionV3.h5
def InceptionV3():
      return(load_model('model_InceptionV3.h5')) 
def lastlayer_en():
      return(load_model('model_VGG16_Xception_ResNet152V2_InceptionV3.h5'))

models = [
    {
        'label': 'VGG16',
        'model': VGG16(),
    },
    {
        'label': 'Xception',
        'model':Xception(),
    },
    {
        'label': 'ResNet152V2',
        'model':ResNet152V2(),
    },
    {
        'label': 'InceptionV3',
        'model':InceptionV3(),
    },
    {
        'label': 'VGG16_Xception_ResNet152V2_InceptionV3',
        'model':lastlayer_en(),
    }]
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
plt.figure()
    
    # Below for loop iterates through your models list
for m in models:
        model = m['model'] # select the model
        #model.fit(X_train, y_train) # train the model
        y_pred=model.predict(test_set) # predict the test data
    # Compute False postive rate, and True positive rate
        #fpr, tpr, thresholds = metrics.roc_curve(y_test, model.y_pred_bin(X_test)[:,1])    model.predict_on_batch(X_test)
        fpr, tpr, thresholds = metrics.roc_curve(test_set.labels, model.predict(test_set))
    # Calculate Area under the curve to display on the plot
        auc = metrics.roc_auc_score(test_set.labels,model.predict(test_set))
    # Now, plot the computed values
        plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))
    # Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

In [149]:
from tensorflow.keras.models import load_model
def VGG16():
      return (load_model('model_VGG16.h5'))  #model_DenseNet_30_09596.h5   #model_EfNet_30_09596.h5    /content/model_VGG16.h5
def Xception():
      return(load_model('model_Xception.h5'))    #model_MobileNet_30_09596.h5  /content/model_Xception.h5
def ResNet152V2():
      return(load_model('model_ResNet152V2.h5'))   #/content/model_ResNet152V2.h5
def InceptionV3():
      return(load_model('model_InceptionV3.h5')) 
#def ():
#      return(load_model('model_VGG16_Xception_ResNet152V2.h5'))

models = [
    {
        'label': 'VGG16',
        'model': VGG16(),
    },
    {
        'label': 'Xception',
        'model':Xception(),
    },
    {
        'label': 'ResNet152V2',
        'model':ResNet152V2(),
    },
    {
        'label': 'InceptionV3',
        'model':InceptionV3(),
    }]
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
plt.figure()
    
    # Below for loop iterates through your models list
for m in models:
        model = m['model'] # select the model
        #model.fit(X_train, y_train) # train the model
        y_pred=model.predict(test_set) # predict the test data
    # Compute False postive rate, and True positive rate
        #fpr, tpr, thresholds = metrics.roc_curve(y_test, model.y_pred_bin(X_test)[:,1])    model.predict_on_batch(X_test)
        fpr, tpr, thresholds = metrics.roc_curve(test_set.labels, model.predict(test_set))
    # Calculate Area under the curve to display on the plot
        auc = metrics.roc_auc_score(test_set.labels,model.predict(test_set))
    # Now, plot the computed values
        plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))
    # Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

In [150]:
from sklearn import metrics
from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc, roc_curve, recall_score, 
                             classification_report, f1_score, average_precision_score, precision_recall_fscore_support)

In [151]:
# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_lr, tpr_lr, label='CNN (area = %0.3f)' % roc_auc_lr)
plt.plot(fpr_VGG16, tpr_VGG16, label='VGG16 (area = %0.3f)' % roc_auc_VGG16)
plt.plot(fpr_Xception, tpr_Xception, label='Xception (area = %0.3f)' % roc_auc_Xception)
plt.plot(fpr_ResNet152V2, tpr_ResNet152V2, label='ResNet152V2 (area = %0.3f)' % roc_auc_ResNet152V2)
plt.plot(fpr_InceptionV3, tpr_InceptionV3, label='InceptionV3 (area = %0.3f)' % roc_auc_InceptionV3)
plt.plot(fpr_VGG16_Xception_ResNet152V2_InceptionV3, tpr_VGG16_Xception_ResNet152V2_InceptionV3, label='Ensemble at last layer (area = %0.3f)' % roc_auc_VGG16_Xception_ResNet152V2_InceptionV3)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves from the investigated models')
plt.legend(loc='best')
plt.show()

In [152]:
# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
#plt.plot(fpr_lr, tpr_lr, label='CNN (area = %0.3f)' % roc_auc_lr)
plt.plot(fpr_VGG16, tpr_VGG16, label='VGG16 (area = %0.3f)' % roc_auc_VGG16)
plt.plot(fpr_Xception, tpr_Xception, label='Xception (area = %0.3f)' % roc_auc_Xception)
plt.plot(fpr_ResNet152V2, tpr_ResNet152V2, label='ResNet152V2 (area = %0.3f)' % roc_auc_ResNet152V2)
plt.plot(fpr_InceptionV3, tpr_InceptionV3, label='InceptionV3 (area = %0.3f)' % roc_auc_InceptionV3)
plt.plot(fpr_VGG16_Xception_ResNet152V2_InceptionV3, tpr_VGG16_Xception_ResNet152V2_InceptionV3, label='Ensemble at last layer (area = %0.3f)' % roc_auc_VGG16_Xception_ResNet152V2_InceptionV3)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves from the investigated models')
plt.legend(loc='best')
plt.show()

In [153]:
# model2= Model(inputs=model1.input, outputs=model1.get_layer(layer_name).output)

model_VGG16_nohead = Model(inputs=model_VGG16.input, outputs=model_VGG16.get_layer("dense_head_vgg").output)
model_Xception_nohead = Model(inputs=model_Xception.input, outputs=model_Xception.get_layer("dense_head_xception").output)
model_ResNet152V2_nohead = Model(inputs=model_ResNet152V2.input, outputs=model_ResNet152V2.get_layer("dense_head_resnet_v2").output)
model_InceptionV3_nohead = Model(inputs=model_InceptionV3.input, outputs=model_InceptionV3.get_layer("dense_head_InceptionV3").output)

for layer in model_VGG16_nohead.layers:
  layer.trainable = False

for layer in model_Xception_nohead.layers:
  layer.trainable = False

for layer in model_ResNet152V2_nohead.layers:
  layer.trainable = False

for layer in model_InceptionV3_nohead.layers:
  layer.trainable = False

visible = tf.keras.layers.Input(shape=(224, 224, 3), name="input")
x = model_VGG16_nohead(visible)
x = model_Xception_nohead(visible)
x = model_ResNet152V2_nohead(visible)
x = model_InceptionV3_nohead(visible)

merge = tf.keras.layers.concatenate([x, x, x, x], name="concatallprobs")
model_out = tf.keras.layers.Dense(1, activation='sigmoid', name="predictions")(merge)

model_VGG16_Xception_ResNet152V2_InceptionV3_nohead = Model(inputs=visible, outputs=model_out)
model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy']) #categorical_smooth_loss
model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.summary()

In [154]:
#Model Parameters
#epochs = 2 # make it 1000

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5,
                                                     restore_best_weights=True,verbose=1) #check val_loss

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=1e-5 ,verbose=1) #min_lr as input for fine tuning next

In [155]:
history = model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.fit(
    aug_train_set,
    steps_per_epoch=TRAIN_IMG_COUNT // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=vali_set,
    validation_steps=VAL_IMG_COUNT // BATCH_SIZE,
    class_weight=class_weight,
    callbacks=[early_stopping_cb, reduce_lr]
)
#y_pred = model_MobileNet_DenseNet_EfNet.predict(test_set)
y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead = model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.predict(test_set)
y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead = y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead > 0.5
fpr_VGG16_Xception_ResNet152V2_InceptionV3_nohead, tpr_VGG16_Xception_ResNet152V2_InceptionV3_nohead, thresholds_VGG16_Xception_ResNet152V2_InceptionV3_nohead = roc_curve(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead)
roc_auc_VGG16_Xception_ResNet152V2_InceptionV3_nohead = auc(fpr_VGG16_Xception_ResNet152V2_InceptionV3_nohead, tpr_VGG16_Xception_ResNet152V2_InceptionV3_nohead)
precision_VGG16_Xception_ResNet152V2_InceptionV3_nohead, recall_VGG16_Xception_ResNet152V2_InceptionV3_nohead, th_VGG16_Xception_ResNet152V2_InceptionV3_nohead = precision_recall_curve(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead)

In [156]:
model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.save("model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.h5")

In [157]:
#Plot training and validation Loss
fig, axarr = plt.subplots(1,3, figsize=(13,5),sharex=True)

sns.set(style="ticks", font_scale = 1)
sns.despine(top=True, right=True, left=False, bottom=False)

historyDF = pd.DataFrame.from_dict(history.history)
ax = sns.lineplot(x =historyDF.index, y = history.history['accuracy'],ax=axarr[0],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_accuracy'],ax=axarr[0],label="VAL");
ax.set_ylabel('accuracy')
ax = sns.lineplot(x =historyDF.index, y = history.history['loss'],ax=axarr[1],label="Training");
ax = sns.lineplot(x =historyDF.index, y = history.history['val_loss'],ax=axarr[1],label="VAL");
ax.set_ylabel('Loss')
ax = sns.lineplot(x =historyDF.index, y = history.history['lr'],ax=axarr[2]);
ax.set_ylabel('Learning Rate')    
axarr[0].set_title("Training and VAL Set - Metric acuuracy")
axarr[1].set_title("Training and VAL Set - Loss")
axarr[2].set_title("Learning Rate during Training")

for ax in axarr:
    ax.set_xlabel('Epochs')
    
plt.suptitle('Training Performance Plots',fontsize=16, weight = 'bold');
fig.tight_layout(pad=3.0)      
plt.show()

In [158]:
loss, acc = model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.evaluate(test_set)

In [159]:
#Predicted labels
prediction_VGG16_Xception_ResNet152V2_InceptionV3_nohead = model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.predict(test_set)
prediction_VGG16_Xception_ResNet152V2_InceptionV3_nohead = prediction_VGG16_Xception_ResNet152V2_InceptionV3_nohead > 0.5

#True labels   VGG16_Xception_ResNet152V2_InceptionV3_nohead
orig_VGG16_Xception_ResNet152V2_InceptionV3_nohead = test_set.labels

In [160]:
cm_VGG16_Xception_ResNet152V2_InceptionV3_nohead = confusion_matrix(orig_VGG16_Xception_ResNet152V2_InceptionV3_nohead, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead)
print('Confusion matrix:')
print(cm_VGG16_Xception_ResNet152V2_InceptionV3_nohead)
print('')

cr_VGG16_Xception_ResNet152V2_InceptionV3_nohead = classification_report(orig_VGG16_Xception_ResNet152V2_InceptionV3_nohead, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead)
print('Classification report:')
print(cr_VGG16_Xception_ResNet152V2_InceptionV3_nohead)
print('')

In [161]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_VGG16_Xception_ResNet152V2_InceptionV3_nohead,display_labels=['NORMAL','PNEUMONIA'])
disp.plot()

In [162]:
err=[i for i, (x, y) in enumerate(zip(y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead, test_set.labels)) if x != y] #check where orig and predictions don't match (wrong classification)
#print(err)
print('errori commessi: ',len(err))

plt.figure(figsize=(10,10))
plt.suptitle("WRONG PREDICTIONS")
for n in range(16): #plot max 16 wrong predictions
    bn=err[n]//BATCH_SIZE #number of batch of wrong prediction train_set[bn][0][image]
    ib=err[n]-BATCH_SIZE*bn #number of the image in the batch size train_set[bn][0][ib]
    #print('id_err:',err[n],' batch:',bn,' diff:',err[n]-BATCH_SIZE*bn)
    ax = plt.subplot(4,4,n+1)
    plt.imshow(test_set[bn][0][ib].astype(np.uint8))  #uint8 to plot
    
    if test_set[bn][1][ib]: #real value
        plt.title("Real: PNEUMONIA")
    else:
        plt.title("Real: NORMAL")
    plt.axis("off")

In [163]:
#Accuracy Result
acc = accuracy_score(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead)
#Precision, Recall and F-Score (For the whole dataset)
results_all = precision_recall_fscore_support(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead, average='macro',zero_division = 1)
#Precision, Recall and F-Score (For each Class)
results_class = precision_recall_fscore_support(test_set.labels, y_pred_prob_VGG16_Xception_ResNet152V2_InceptionV3_nohead, average=None, zero_division = 1)

#Organise the Results into a Dataframe
metric_columns = ['Precision','Recall', 'F-Score','S']
all_df = pd.concat([pd.DataFrame(list(results_class)).T,pd.DataFrame(list(results_all)).T])
all_df.columns = metric_columns
all_df.index = ['NORMAL', 'PNEUMONIA', 'Total']

#Function to plot the metrics into a nice bar Plot
def metrics_plot(df,metric):
    plt.figure(figsize=(22,10))
    ax = sns.barplot(data =df, x=df.index, y = metric,palette = "Blues_d")
    #Bar Labels
    for p in ax.patches:
        ax.annotate("%.1f%%" % (100*p.get_height()), (p.get_x() + p.get_width() / 2., abs(p.get_height())),
        ha='center', va='bottom', color='black', xytext=(-3, 5),rotation = 'horizontal',textcoords='offset points')
    sns.despine(top=True, right=True, left=True, bottom=False)
    ax.set_xlabel('Class',fontsize = 14,weight = 'bold')
    ax.set_ylabel(metric,fontsize = 14,weight = 'bold')
    ax.set(yticklabels=[])
    ax.axes.get_yaxis().set_visible(False) 
    plt.title(metric+ ' Results per Class', fontsize = 16,weight = 'bold');
    
metrics_plot(all_df, 'Precision')#Results by Class
metrics_plot(all_df, 'Recall')#Results by Class
metrics_plot(all_df, 'F-Score')#Results by Class
print('**Overall Results**')
print('Accuracy Result: %.2f%%'%(acc*100)) #Accuracy of the whole Dataset
print('Precision Result: %.2f%%'%(all_df.iloc[2,0]*100))#Precision of the whole Dataset
print('Recall Result: %.2f%%'%(all_df.iloc[2,1]*100))#Recall of the whole Dataset
print('F-Score Result: %.2f%%'%(all_df.iloc[2,2]*100))#F-Score of the whole Dataset

In [177]:
from tensorflow.keras.models import load_model
def VGG16():
      return (load_model('model_VGG16.h5'))  #model_DenseNet_30_09596.h5   #model_EfNet_30_09596.h5    /content/model_VGG16.h5
def Xception():
      return(load_model('model_Xception.h5'))    #model_MobileNet_30_09596.h5  /content/model_Xception.h5
def ResNet152V2():
      return(load_model('model_ResNet152V2.h5')) #./model_VGG16_Xception_ResNet152V2_InceptionV3.h5
def InceptionV3():
      return(load_model('model_InceptionV3.h5')) 
def lastlayer_en():
      return(load_model('model_VGG16_Xception_ResNet152V2_InceptionV3.h5'))

def withoutheads_en():
      return(load_model('model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.h5'))
    #./model_VGG16_Xception_ResNet152V2_InceptionV3_nohead.h5
    
models = [
    {
        'label': 'VGG16',
        'model': VGG16(),
    },
    {
        'label': 'Xception',
        'model':Xception(),
    },
    {
        'label': 'ResNet152V2',
        'model':ResNet152V2(),
    },
    {
        'label': 'InceptionV3',
        'model':InceptionV3(),
    },
    {
        'label': 'lastlayer en',
        'model':lastlayer_en(),
    },
    {
        'label': 'en without heads',
        'model':withoutheads_en(),
     }]
from sklearn import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
plt.figure()
    
    # Below for loop iterates through your models list
for m in models:
        model = m['model'] # select the model
        #model.fit(X_train, y_train) # train the model
        y_pred=model.predict(test_set) # predict the test data
    # Compute False postive rate, and True positive rate
        #fpr, tpr, thresholds = metrics.roc_curve(y_test, model.y_pred_bin(X_test)[:,1])    model.predict_on_batch(X_test)
        fpr, tpr, thresholds = metrics.roc_curve(test_set.labels, model.predict(test_set))
    # Calculate Area under the curve to display on the plot
        auc = metrics.roc_auc_score(test_set.labels,model.predict(test_set))
    # Now, plot the computed values
        plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % (m['label'], auc))
    # Custom settings for the plot 
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()   # Display

In [176]:
# Plot ROC curve
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_lr, tpr_lr, label='CNN (area = %0.3f)' % roc_auc_lr)
plt.plot(fpr_VGG16, tpr_VGG16, label='VGG16 (area = %0.3f)' % roc_auc_VGG16)
plt.plot(fpr_Xception, tpr_Xception, label='Xception (area = %0.3f)' % roc_auc_Xception)
plt.plot(fpr_ResNet152V2, tpr_ResNet152V2, label='ResNet152V2 (area = %0.3f)' % roc_auc_ResNet152V2)
plt.plot(fpr_InceptionV3, tpr_InceptionV3, label='InceptionV3 (area = %0.3f)' % roc_auc_InceptionV3)
plt.plot(fpr_VGG16_Xception_ResNet152V2_InceptionV3, tpr_VGG16_Xception_ResNet152V2_InceptionV3, label='Ensemble at last layer (area = %0.3f)' % roc_auc_VGG16_Xception_ResNet152V2_InceptionV3)
plt.plot(fpr_VGG16_Xception_ResNet152V2_InceptionV3_nohead, tpr_VGG16_Xception_ResNet152V2_InceptionV3_nohead, label='Ensemble without heads (area = %0.3f)' % roc_auc_VGG16_Xception_ResNet152V2_InceptionV3_nohead)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curves from the investigated models')
plt.legend(loc='best')
plt.show()